# Exercise 1

### Necessary imports

In [50]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from statistics import mean
from IPython.display import display, HTML
from nltk import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sentence_transformers import SentenceTransformer, util

tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')

### Dataset Loading

In [51]:
df = pd.read_csv('data/TLN-definitions-23.tsv', sep='\t')
df_door = df['door']
df_ladybug = df['ladybug']
df_pain = df['pain']
df_blurriness = df['blurriness']

### Pre-processing of dataset

In [52]:
# Stop words removal
df_door_nostop = df_door.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_ladybug_nostop = df_ladybug.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_pain_nostop = df_pain.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df_blurriness_nostop = df_blurriness.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Stemming and tokenization (with puntuaction removal)
ps = PorterStemmer()
df_door_stem = df_door_nostop.apply(
    lambda x: tokenizer.tokenize(' '.join([ps.stem(word) for word in x.split()]).lower())
)
df_ladybug_stem = df_ladybug_nostop.apply(
    lambda x: tokenizer.tokenize(' '.join([ps.stem(word) for word in x.split()]).lower())
)
df_pain_stem = df_pain_nostop.apply(
    lambda x: tokenizer.tokenize(' '.join([ps.stem(word) for word in x.split()]).lower())
)
df_blurriness_stem = df_blurriness_nostop.apply(
    lambda x: tokenizer.tokenize(' '.join([ps.stem(word) for word in x.split()]).lower())
)

# Lemmatization and tokenization (with puntuaction removal)
lemmatizer = WordNetLemmatizer()
df_door_lem = df_door_nostop.apply(
    lambda x: tokenizer.tokenize(' '.join([lemmatizer.lemmatize(word) for word in x.split()]).lower())
)
df_ladybug_lem = df_ladybug_nostop.apply(
    lambda x: tokenizer.tokenize(' '.join([lemmatizer.lemmatize(word) for word in x.split()]).lower())
)
df_pain_lem = df_pain_nostop.apply(
    lambda x: tokenizer.tokenize(' '.join([lemmatizer.lemmatize(word) for word in x.split()]).lower())
)
df_blurriness_lem = df_blurriness_nostop.apply(
    lambda x: tokenizer.tokenize(' '.join([lemmatizer.lemmatize(word) for word in x.split()]).lower())
)

## First Method: Overlap of words in the definitions

### Function for calculating the overlap between the definitions of two words

In [53]:
def calculate_overlap(sentence_1: list, sentence_2: list) -> int:
    return len(set(sentence_1).intersection(set(sentence_2)))

In [54]:
def definitions_similarity(definitions: list) -> float:
    similarity_matrix = []
    for sentence_1 in definitions:
        for sentence_2 in definitions:
            if sentence_1 == sentence_2:
                continue
            overlap = calculate_overlap(sentence_1, sentence_2)
            overlap = overlap / (len(sentence_1) if len(sentence_1) < len(sentence_2) else len(sentence_2))
            similarity_matrix.append(overlap)
    return round(mean(similarity_matrix), 2)

In [55]:
# Overlap between all definitions of door, ladybug, pain and blurriness with both stemming and lemmatization
door_similarity_stem = definitions_similarity(df_door_stem)
ladybug_similarity_stem = definitions_similarity(df_ladybug_stem)
pain_similarity_stem = definitions_similarity(df_pain_stem)
blurriness_similarity_stem = definitions_similarity(df_blurriness_stem)

door_similarity_lem = definitions_similarity(df_door_lem)
ladybug_similarity_lem = definitions_similarity(df_ladybug_lem)
pain_similarity_lem = definitions_similarity(df_pain_lem)
blurriness_similarity_lem = definitions_similarity(df_blurriness_lem)

# I create the dataframe for printing the table with the results of the similarity with stemming

data_stem =  [['pain\n {:.2f}'.format(pain_similarity_stem), 'door\n {:.2f}'.format(door_similarity_stem)], 
              ['bluriness\n {:.2f}'.format(blurriness_similarity_stem), 'ladybug\n {:.2f}'.format(ladybug_similarity_stem)]]
df_similarity_stem = pd.DataFrame(data_stem, columns=['Abstract', 'Concrete'], index=['Generic', 'Specific'])

print("Similarity between all definitions with Stemming")
display(HTML(df_similarity_stem.to_html().replace("\\n","<br>")))

# I create the dataframe for printing the table with the results of the similarity with lemmatization

data_lem =  [['pain\n {:.2f}'.format(pain_similarity_lem), 'door\n {:.2f}'.format(door_similarity_lem)],
             ['bluriness\n {:.2f}'.format(blurriness_similarity_lem), 'ladybug\n {:.2f}'.format(ladybug_similarity_lem)]]
df_similarity_lem = pd.DataFrame(data_lem, columns=['Abstract', 'Concrete'], index=['Generic', 'Specific'])

print("Similarity between all definitions with Lemmatization")
display(HTML(df_similarity_lem.to_html().replace("\\n","<br>")))

Similarity between all definitions with Stemming


,Abstract,Concrete
Generic,pain 0.22,door 0.20
Specific,bluriness 0.08,ladybug 0.56


Similarity between all definitions with Lemmatization


,Abstract,Concrete
Generic,pain 0.20,door 0.19
Specific,bluriness 0.08,ladybug 0.53


## Second Method: Similarity of definitions with CountVectorizer

### Pre-processing

In [56]:
# Lemmatization
cv_door_lem = df_door_nostop.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]).lower())
cv_ladybug_lem = df_ladybug_nostop.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]).lower())
cv_pain_lem = df_pain_nostop.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]).lower())
cv_blurriness_lem = df_blurriness_nostop.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]).lower())

### Obtaining the CountVectorizer vectors

In [57]:
# Obtaining the CountVectorizer
vectorizer = CountVectorizer()
vectors_door = vectorizer.fit_transform(cv_door_lem)
vectors_ladybug = vectorizer.fit_transform(cv_ladybug_lem)
vectors_pain = vectorizer.fit_transform(cv_pain_lem)
vectors_blurriness = vectorizer.fit_transform(cv_blurriness_lem)

### Calculating the cosine similarity between the vectors of the same word

In [58]:
def vectors_similarity(vectors: list) -> float:
    similarity_matrix = []
    for sentence_1 in vectors:
        for sentence_2 in vectors:
            if np.array_equal(sentence_1, sentence_2):
                continue
            similarity_matrix.append(distance.cosine(sentence_1, sentence_2))
    return round(1 - mean(similarity_matrix), 2)

In [59]:
# Cosine similarity between all definitions of door, ladybug, pain and blurriness
door_similarity = vectors_similarity(vectors_door.toarray())
ladybug_similarity = vectors_similarity(vectors_ladybug.toarray())
pain_similarity = vectors_similarity(vectors_pain.toarray())
blurriness_similarity = vectors_similarity(vectors_blurriness.toarray())

# I create the dataframe for printing the table with the results of the similarity with CountVectorizer
data_cv = [['pain\n {:.2f}'.format(pain_similarity), 'door\n {:.2f}'.format(door_similarity)],
              ['bluriness\n {:.2f}'.format(blurriness_similarity), 'ladybug\n {:.2f}'.format(ladybug_similarity)]]
df_similarity_cv = pd.DataFrame(data_cv, columns=['Abstract', 'Concrete'], index=['Generic', 'Specific'])
print("Similarity between all definitions with CountVectorizer")
display(HTML(df_similarity_cv.to_html().replace("\\n","<br>")))

Similarity between all definitions with CountVectorizer


,Abstract,Concrete
Generic,pain 0.14,door 0.13
Specific,bluriness 0.05,ladybug 0.43


## Third Method (State of Art): Sentence Transformers

In [60]:
def calculate_similarity_tran(dataset: list) -> float:
    # I load the transformer model
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    # I compute the embeddings for the dataset
    embeddings = model.encode(dataset, convert_to_tensor=True)

    # I compute the cosine similarity between all the embeddings and I return the mean
    return round(util.torch.mean(util.cos_sim(embeddings, embeddings)).item(), 2)

In [62]:
# Cosine similarity between all definitions of door, ladybug, pain and blurriness
door_similarity_tran = calculate_similarity_tran(df_door)
ladybug_similarity_tran = calculate_similarity_tran(df_ladybug)
pain_similarity_tran = calculate_similarity_tran(df_pain)
blurriness_similarity_tran = calculate_similarity_tran(df_blurriness)

# I create the dataframe for printing the table with the results of the similarity with Transformer
data_tran = [['pain\n {:.2f}'.format(pain_similarity_tran), 'door\n {:.2f}'.format(door_similarity_tran)],
              ['bluriness\n {:.2f}'.format(blurriness_similarity_tran), 'ladybug\n {:.2f}'.format(ladybug_similarity_tran)]]
df_similarity_tran = pd.DataFrame(data_tran, columns=['Abstract', 'Concrete'], index=['Generic', 'Specific'])
print("Similarity between all definitions with SentenceTransformer")
display(HTML(df_similarity_tran.to_html().replace("\\n","<br>")))

Similarity between all definitions with SentenceTransformer


,Abstract,Concrete
Generic,pain 0.60,door 0.50
Specific,bluriness 0.42,ladybug 0.71
